<a href="https://colab.research.google.com/github/WhiteChocolate0/ProgramApplication/blob/main/EX05_04_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## EX05-04 Reply Template Messages
* 依使用者輸入：confirm, buttons, carousel, imgCarousel，分別傳送確認模板、按鈕模板、輪播模板及圖片輪播模板。
* 模板上的按鈕能觸發不同類型的 Action Object。

In [2]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 9.0 MB/s eta 0:00:00


In [5]:
# for colab
from flask import Flask, request, abort
from flask_ngrok import run_with_ngrok
from google.colab import userdata

from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
    TemplateSendMessage, ConfirmTemplate, MessageAction,
    CarouselTemplate, CarouselColumn
)

app = Flask(__name__)
run_with_ngrok(app)

line_bot_api = LineBotApi(userdata.get("LINE_CHANNEL_ACCESS_TOKEN"))
handler = WebhookHandler(userdata.get("LINE_CHANNEL_SECRET"))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    msg = event.message.text.lower()

    # Confirm 模板
    if msg == "confirm":
        confirm_template = TemplateSendMessage(
            alt_text='這是確認模板',
            template=ConfirmTemplate(
                text='你喜歡吉卜力動畫嗎？',
                actions=[
                    MessageAction(label='喜歡', text='我超愛龍貓！'),
                    MessageAction(label='不喜歡', text='我比較喜歡皮克斯'),
                ]
            )
        )
        line_bot_api.reply_message(event.reply_token, confirm_template)

    # Carousel 模板
    elif msg == "carousel":
        carousel_template = TemplateSendMessage(
            alt_text='這是輪播模板',
            template=CarouselTemplate(columns=[
                CarouselColumn(
                    thumbnail_image_url='https://upload.wikimedia.org/wikipedia/zh/7/7d/Summer_Wars_poster.jpg',
                    title='夏日大作戰',
                    text='細田守經典之作',
                    actions=[
                        MessageAction(label='介紹', text='這是一部描述虛擬世界的動畫電影。')
                    ]
                ),
                CarouselColumn(
                    thumbnail_image_url='https://upload.wikimedia.org/wikipedia/zh/thumb/4/4f/Castle_of_Cagliostro_poster.png/250px-Castle_of_Cagliostro_poster.png',
                    title='卡里奧斯特羅城',
                    text='宮崎駿第一部導演電影',
                    actions=[
                        MessageAction(label='介紹', text='這是一部經典魯邦三世電影！')
                    ]
                )
            ])
        )
        line_bot_api.reply_message(event.reply_token, carousel_template)

    # 其他訊息 → Echo 回傳
    else:
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=f'你說了：{msg}')
        )

if __name__ == "__main__":
    app.run()

<ipython-input-5-7f8d5f3d03c0>:19: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(userdata.get("LINE_CHANNEL_ACCESS_TOKEN"))


 * Serving Flask app '__main__'
 * Debug mode: off


<ipython-input-5-7f8d5f3d03c0>:20: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(userdata.get("LINE_CHANNEL_SECRET"))
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
